In [194]:
import pandas as pd
from os import path
import os

DATA_IN = "../data/raw/semi-raw_data.csv"
assert path.exists(DATA_IN)

df = pd.read_csv(DATA_IN, index_col=0)

In [195]:
df.columns

Index(['OBJECTID', 'UNITID', 'SIGID', 'MAINSTREET', 'CROSSSTREE', 'OWNER2',
       'ROUTE', 'MILEPOINT', 'DESCRIPTIO', 'TIMES', 'OWNER', 'TYPE', 'INTID',
       'ATMSID', 'geometry'],
      dtype='object')

In [203]:
renames = {
    'CROSSSTREE': 'cross_street',
    'DESCRIPTIO': 'desription',
    'INTID': 'int_ID',
    'MAINSTREET': 'main_street',
    'MILEPOINT': 'milepoint',
    'OBJECTID': 'row_ID',
    'OWNER': 'owner',
    'ROUTE': 'route',
    'SIGID': 'sig_ID',
    'TIMES': 'times',
    'TYPE': 'type',
    'UNITID': 'unit_ID'
}

drops = ['OWNER2']

def clean_columns(df):
    df = df.drop(drops, axis=1)
    df = df.rename(renames, axis=1)
    return df

clean_columns(df).head()
df.head()


,OBJECTID,UNITID,SIGID,MAINSTREET,CROSSSTREE,OWNER2,ROUTE,MILEPOINT,DESCRIPTIO,TIMES,OWNER,TYPE,INTID,ATMSID,geometry
0,1,MPW017206,SIG017206,OLD HENRY,I 265 RAMP,S,KY-3084,1.250,OLD HENRY @ I 265 RAMP,24 HOURS,KYTC,1,7674756999,NaN,POINT (-85.4993371456453 38.27063209239155)
1,2,MPW017211,SIG017211,SHELBYVILLE,BECKLEY WOODS,S,US 60,12.291,SHELBYVILLE @ BECKLEY WOODS,24 HOURS,KYTC,1,7895940672,NaN,POINT (-85.49847895741786 38.241589472827904)
2,3,MPW017290,SIG017290,SHELBYVILLE,LAKE FOREST,S,US-60,12.580,SHELBYVILLE @ LAKE FOREST,24 HOURS,KYTC,1,12375940763,NaN,POINT (-85.4935498603372 38.24051723243606)
3,4,MPW015159,SIG015159,BROWNSBORO,I 265 RAMP,S,KY-22,3.700,BROWNSBORO @ I 265 RAMP,24 HOURS,KYTC,1,14390724999,NaN,POINT (-85.57021228272409 38.31100207202721)
4,5,MPW011726,SIG011726,HERR,LIME KILN,S,KY-22,0.440,HERR @ LIME KILN,24 HOURS,KYTC,1,18422901386,NaN,POINT (-85.62796526747577 38.28391602421024)


In [248]:
df['TIMES']

0       24 HOURS
1       24 HOURS
2       24 HOURS
3       24 HOURS
4       24 HOURS
          ...   
1086    24 HOURS
1087    24 HOURS
1088    24 HOURS
1089    24 HOURS
1090         NaN
Name: TIMES, Length: 1091, dtype: object

In [247]:
df[df.TIMES == 'ACTUATED']
# TYPE == 4 # Hmm

,OBJECTID,UNITID,SIGID,MAINSTREET,CROSSSTREE,OWNER2,ROUTE,MILEPOINT,DESCRIPTIO,TIMES,OWNER,TYPE,INTID,ATMSID,geometry
1031,1032,MPW004166,SIG004166,BRECKINRIDGE,DIXIE,M,NaN,NaN,BRECKINRIDGE @ DIXIE,ACTUATED,METRO,4,131280658161,NaN,POINT (-85.78365846309039 38.24506461198751)


In [201]:
df.INTID.value_counts(), df.DESCRIPTIO.value_counts()

(INTID
 2               23
 218321612354     3
 160670384393     3
 167401885701     2
 137970384061     2
                 ..
 41854492713      1
 43662901713      1
 50802930562      1
 52900723408      1
 121707553999     1
 Name: count, Length: 1048, dtype: int64,
 DESCRIPTIO
 DIXIE @ KERRICK                  3
 HURSTBOURNE @ I 64 RAMP          3
 BARDSTOWN @ LONGEST              3
 BARDSTOWN @ BONNYCASTLE          3
 OLD HENRY @ I 265 RAMP           2
                                 ..
 WILDWOOD @ SHELBYVILLE           1
 LA GRANGE @ HURSTBOURNE RAMP     1
 HERMITAGE @ WESTPORT             1
 FOX HARBOR @ U S HIGHWAY 42      1
 BEULAH CHURCH RD @ I 265 RAMP    1
 Name: count, Length: 1039, dtype: int64)

In [202]:
df[df.INTID != "2"][['INTID', 'SIGID']]
# index 1087 INTID has an E in the number 2834756E294

,INTID,SIGID
0,7674756999,SIG017206
1,7895940672,SIG017211
2,12375940763,SIG017290
3,14390724999,SIG015159
4,18422901386,SIG011726
...,...,...
1086,82280188999,SIG000001
1087,2834756E294,SIG000002
1088,69867132999,SIG000003
1089,97890518999,SIG000004
